## Demo notebook to check out the Google Places API and the results

In [1]:
import requests
import openai

### Define a function

In [3]:
#get nearby restaurants
def get_nearby_restaurants(api_key, location, radius=200, keyword='restaurant', num_results=5):
    '''
    Returns the restaurants from the API call based on location, radius
    '''
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    params = {
        'location': location,
        'radius': radius,
        'keyword': keyword,
        'key': api_key
    }
    
    response = requests.get(base_url, params=params)
    results = response.json().get('results', [])

    # Limit the number of results to the top 'num_results'
    results = results[:num_results]
    
    return results


def get_place_details(api_key, place_id):
    '''
    Returns details on a place defined by the place_id, e.g. reviews
    '''
    base_url = 'https://maps.googleapis.com/maps/api/place/details/json'

    params = {
        'place_id': place_id,
        'key': api_key
    }

    response = requests.get(base_url, params=params)
    result = response.json().get('result', {})

    return result


def generate_summary(reviews):
    #Concatenate all reviews
    review_text = "\n".join(review.get('text', '') for review in reviews)

    #Define the prompt for GPT
    prompt = f"Summarize the reviews for this place:\n{review_text}"

    #Generate short summary
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokes=100
    )

    #Extract the generated summary from GPT's response
    summary = response['choices'][0]['text'].strip()

    return summary
    

In [4]:
# Replace 'YOUR_API_KEY' with your actual Google Places API key
api_key = 'YOUR API KEY'
openai.api_key = 'YOUR OPENAI_API_KEY'

# Munich center coordinates (for example)
munich_center = '48.1363964,11.5609501'

# Set radius and keyword
restaurants = get_nearby_restaurants(api_key, munich_center, radius=200, keyword='restaurant', num_results=5)

# Print the names of the found restaurants
for restaurant in restaurants:
    #Get place details 
    restaurant_id = restaurant.get('place_id')
    place_details = get_place_details(api_key, restaurant_id)

    

    #Get reviews for the place
    reviews = place_details.get('reviews', [])


    #Generate a summary
    summary = generate_summary(reviews)

    #Print Restaurant name and average rating
    print("Restaurant name: ", restaurant.get('name'))
    print("Rating:  ", restaurant.get('rating'))
    print(f"Summary: {summary}\n")

    
    # for i, review in enumerate(reviews[:5]):
    #     print(f"Review {i + 1}: {review.get('text')}")

    # print('\n')

Restaurant name:  Nur Restaurant
Rating:   3.9
Review 1: Iraki restaurant
Quick service
Good quality
Review 2: Used to come here a lot! Kebabs were juicy, everything was really good. It’s like that place never existed. Dirty (mold in the oil), and food tastes spoiled: I had to apologize to my friend to whom I extensively asked to come here. A shame.
Review 3: 2/5 Food
2/5 Service
2/5 Vibes
3/5 Prices

Another Arab restaurant which is close to the Train Central Station which offers a big variety of -low-to-medium- quality Arab dishes. Be carful with the prices, I got different price check compared to the prices at the menu and when I asked, the waitress said we’ve not updated our menu yet, even, they did it manually 🫣
Review 4: Food is delicious; ordered hummus and grilled skewers. Everything is fresh made, especially the nan, which comes with the food. A must try if you are near the train station.
Review 5: Food was decent, good portion compared to the price. The service was good, no c